In [14]:
import gensim
import pandas as pd
import numpy as np

from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [15]:
model = gensim.models.KeyedVectors.load("araneum_none_fasttextcbow_300_5_2018.model")

In [16]:
data1 = pd.read_csv('train_1.csv')

In [ ]:
false_deform = pd.read_csv('false_deforming.csv')

In [17]:
with open('forms.csv') as file: # словарь словоформ
    forms1 = file.readlines()
forms1 = [word.strip('\"»«\n)(') for word in forms1]
forms1 = [word.lower() for word in forms1 if word]
labels = pd.Series(0, index=np.arange(len(forms1)))
forms1 = pd.DataFrame({'words': forms1, 'values': labels})

In [ ]:
forms = pd.concat([forms1, false_deform], axis=0).sample(frac=1)
forms.drop(['Unnamed: 0'], axis=1, inplace=True)

In [37]:
df = pd.concat([forms.sample(900), data1], axis=0).sample(frac=1)
df.drop(['Unnamed: 0'], axis=1, inplace=True)

In [38]:
df['model'] = df.words.apply(lambda x: model[x])

In [49]:
X = np.vstack(df.model.values)
y = df['values'].values

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [51]:
clf = LogisticRegression().fit(X_train, y_train)

In [52]:
y_pred = clf.predict(X_test)
f1_score(y_test, y_pred)

0.7935368043087971

In [13]:
dataset = pd.read_csv('words_only.csv', sep='\t', names=['word'])

In [29]:
ff = open('deforming.csv', 'w+')

for word in dataset.word:
    res = clf.predict(model[word].reshape(1,-1))
    if res[0] == 1:
        example = f"{word}\t{res}" + "\n"
        ff.write(example)


In [30]:
examples = pd.read_csv('examples_of_deform_2.csv', sep='\t', names=['word', 'class'])
examples.to_csv('examples_of_deforming_2.csv')
